In [ ]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals

In [ ]:
import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True

In [ ]:
from datetime import date


def q6_pandas_native(line_item_ds):
    var1 = date(1994, 1, 1)
    var2 = date(1995, 1, 1)
    var3 = 0.05
    var4 = 0.07
    var5 = 24

    flineitem = line_item_ds[
        (line_item_ds["l_shipdate"] >= var1)
        & (line_item_ds["l_shipdate"] < var2)
        & (line_item_ds["l_discount"] >= var3)
        & (line_item_ds["l_discount"] <= var4)
        & (line_item_ds["l_quantity"] < var5)
    ]

    result_value = (flineitem["l_extendedprice"] * flineitem["l_discount"]).sum()

    return pd.DataFrame({"revenue": [result_value]})

In [ ]:
from datetime import datetime

import narwhals as nw


def q6(line_item_raw) -> None:
    var_1 = datetime(1994, 1, 1)
    var_2 = datetime(1995, 1, 1)
    var_3 = 24

    line_item_ds = nw.from_native(line_item_raw)

    result = (
        line_item_ds.filter(
            nw.col("l_shipdate").is_between(var_1, var_2, closed="left"),
            nw.col("l_discount").is_between(0.05, 0.07),
            nw.col("l_quantity") < var_3,
        )
        .with_columns((nw.col("l_extendedprice") * nw.col("l_discount")).alias("revenue"))
        .select(nw.sum("revenue"))
    )
    return nw.to_native(result)

In [ ]:
def q6_ibis(lineitem, *, tool: str) -> None:
    var1 = datetime(1994, 1, 1)
    var2 = datetime(1995, 1, 1)
    var3 = 0.05
    var4 = 0.07
    var5 = 24

    q_final = (
        lineitem.filter(
            (lineitem["l_shipdate"] >= var1) & (lineitem["l_shipdate"] < var2)
        )
        .filter((lineitem["l_discount"] >= var3) & (lineitem["l_discount"] <= var4))
        .filter(lineitem["l_quantity"] < var5)
        .mutate(revenue=ibis._["l_extendedprice"] * (ibis._["l_discount"]))
        .agg(revenue=ibis._["revenue"].sum())
    )

    if tool == "pandas":
        return q_final.to_pandas()
    if tool == "polars":
        return q_final.to_polars()
    raise ValueError("expected pandas or polars")

In [ ]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"
region = dir_ + "region.parquet"
nation = dir_ + "nation.parquet"
customer = dir_ + "customer.parquet"
lineitem = dir_ + "lineitem.parquet"
orders = dir_ + "orders.parquet"
supplier = dir_ + "supplier.parquet"
part = dir_ + "part.parquet"
partsupp = dir_ + "partsupp.parquet"

In [ ]:
import ibis

con_pd = ibis.pandas.connect()
con_pl = ibis.polars.connect()

IO_FUNCS = {
    "pandas": lambda x: pd.read_parquet(x, engine="pyarrow"),
    "pandas[pyarrow]": lambda x: pd.read_parquet(
        x, engine="pyarrow", dtype_backend="pyarrow"
    ),
    "pandas[pyarrow][ibis]": lambda x: con_pd.read_parquet(
        x, engine="pyarrow", dtype_backend="pyarrow"
    ),
    "polars[eager]": lambda x: pl.read_parquet(x),
    "polars[lazy]": lambda x: pl.scan_parquet(x),
    "polars[lazy][ibis]": lambda x: con_pl.read_parquet(x),
}

In [ ]:
results = {}

## pandas, pyarrow dtypes, via ibis

In [ ]:
tool = "pandas[pyarrow][ibis]"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6_ibis(fn(lineitem), tool='pandas')
results[tool] = timings.all_runs

## polars, lazy, via ibis

In [ ]:
tool = "polars[lazy][ibis]"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6_ibis(fn(lineitem), tool='polars')
results[tool] = timings.all_runs

## pandas, pyarrow dtypes, native

In [ ]:
tool = "pandas[pyarrow]"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6_pandas_native(fn(lineitem))
results[tool + "[native]"] = timings.all_runs

## pandas via Narwhals

In [ ]:
tool = "pandas"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6(fn(lineitem))
results[tool] = timings.all_runs

## pandas, pyarrow dtypes, via Narwhals

In [ ]:
tool = "pandas[pyarrow]"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6(fn(lineitem))
results[tool] = timings.all_runs

## Polars read_parquet

In [ ]:
tool = "polars[eager]"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6(fn(lineitem))
results[tool] = timings.all_runs

## Polars scan_parquet

In [ ]:
tool = "polars[lazy]"
fn = IO_FUNCS[tool]
timings = %timeit -o -q q6(fn(lineitem)).collect()
results[tool] = timings.all_runs

## Save

In [ ]:
import json

with open("results.json", "w") as fd:
    json.dump(results, fd)